# Final Project Code

### Bailey Blum, Molly Giglia, Horacio Lopez & Ale Lopez

# DONT RUN THIS --------------

In [1]:
import json
import pandas as pd
    

with open('yelp_dataset/review.json') as json_file:      
    data = json_file.readlines()
    # this line below may take at least 8-10 minutes of processing for 4-5 million rows. It converts all strings in list to actual json objects. 
    data = list(map(json.loads, data)) 

In [2]:
texts = [d['text'] for d in data]
stars = [d['stars'] for d in data]

In [3]:
from collections import Counter

freqs = Counter(stars)

# the least common class is the maximum number we want for all classes
max_allowable = 5000
#max_allowable = freqs.most_common()[-1][1]
num_added = {clss: 0 for clss in freqs.keys()}
new_stars = []
new_texts = []
total = 0
counter = 0

for i, y in enumerate(stars):
    total += len(texts[i])
    counter += 1

avg = total / counter

 
def balance_classes(texts, stars):

    freqs = Counter(stars)

    # the least common class is the maximum number we want for all classes
    max_allowable = 5000
    #max_allowable = freqs.most_common()[-1][1]
    num_added = {clss: 0 for clss in freqs.keys()}
    new_stars = []
    new_texts = []
    for i, y in enumerate(stars):
        if len(texts[i]) < avg*1.5 and len(texts[i]) > avg:
            if num_added[y] < max_allowable:
                new_stars.append(y)
                new_texts.append(texts[i])
                num_added[y] += 1
    return new_texts, new_stars

In [4]:
#print(Counter(stars))
balanced_x, balanced_y = balance_classes(texts, stars)
print(Counter(balanced_y))
#print(balanced_y[:10])

Counter({5.0: 5000, 4.0: 5000, 3.0: 5000, 1.0: 5000, 2.0: 5000})


In [5]:
import os
os.remove("texts.txt")
os.remove("stars.txt")
f = open("texts.txt", "w")
f2 = open("stars.txt", "w")
for item in balanced_x:
    f.write(item + "Gigliablurb")
for item in balanced_y:
    f2.write(str(int(item)) + '\n')
f.close()
f2.close()

# RUN FROM HERE DOWN -------------------

In [6]:
import json
import pandas as pd
from collections import Counter

f = open("texts.txt", "r")
f2 = open("stars.txt", "r")

balanced_x = []
balanced_y = []
listOfLines = f.read().split('Gigliablurb')
for item in listOfLines:
    balanced_x.append(item)
for item in f2.readlines():
    balanced_y.append(item)
    
f.close()
f2.close()

balanced_x = balanced_x[:-1]
print(len(balanced_x))

25000


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
import datetime
 
# This vectorizer breaks text into single words and bi-grams
# and then calculates the TF-IDF representation
vectorizer = TfidfVectorizer(ngram_range=(1,2))
t1 = datetime.datetime.now()
 
# the 'fit' builds up the vocabulary from all the reviews
# while the 'transform' step turns each indivdual text into
# a matrix of numbers.
vectors = vectorizer.fit_transform(balanced_x)
print(datetime.datetime.now() - t1)

0:00:12.199060


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vectors, balanced_y, test_size=0.20, random_state=42)

In [9]:
## from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn import tree

# initialise the Naive-Bayes classifier
#classifier = MultinomialNB()
#classifier = ComplementNB()
classifier = LinearSVC()
#classifier = tree.DecisionTreeClassifier(max_depth=20)
#classifier = RandomForestClassifier()
#classifier = AdaBoostClassifier() #takes 7.5 mins

#Run it
t1 = datetime.datetime.now()
#X_train = X_train.toarray()
classifier.fit(X_train, y_train)
print(datetime.datetime.now() - t1)

0:00:04.874257


In [10]:
preds = classifier.predict(X_test)
print(list(preds[:10]))
print(y_test[:10])

['5\n', '4\n', '2\n', '4\n', '5\n', '1\n', '2\n', '4\n', '4\n', '4\n']
['5\n', '2\n', '1\n', '4\n', '4\n', '1\n', '1\n', '4\n', '2\n', '5\n']


In [11]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, preds))


0.5882


In [12]:
from sklearn.metrics import classification_report
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

          1
       0.69      0.78      0.73       976
          2
       0.51      0.46      0.49      1003
          3
       0.49      0.44      0.46       996
          4
       0.51      0.51      0.51      1008
          5
       0.69      0.75      0.72      1017

    accuracy                           0.59      5000
   macro avg       0.58      0.59      0.58      5000
weighted avg       0.58      0.59      0.58      5000



In [13]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, preds))

[[765 167  19   6  19]
 [259 464 217  47  16]
 [ 60 217 437 234  48]
 [ 14  40 186 517 251]
 [ 11  14  26 208 758]]
